In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

/Users/roarke/projects/FIFA Seer/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
emb_players_df = pd.read_csv("emb_players_df.csv")
pd.set_option('display.max_columns', None)
emb_players_df.head()

,short_name,age,height_cm,weight_kg,nationality_emb_0,nationality_emb_1,club_name_emb_0,club_name_emb_1,club_name_emb_2,club_name_emb_3,league_name_emb_0,league_name_emb_1,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33.0,170.0,72.0,5.200541,5.201881,0.923094,0.908502,0.884788,0.884957,4.387769,4.342915,93.0,93.0,67500000.0,560000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5.0,4.0,4.0,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85.0,95.0,70.0,91.0,88.0,96.0,93.0,94.0,91.0,96.0,91.0,80.0,91.0,94.0,95.0,86.0,68.0,72.0,69.0,94.0,44.0,40.0,93.0,95.0,75.0,96.0,35.0,24.0
1,Cristiano Ronaldo,35.0,187.0,83.0,3.258914,3.233033,0.898334,0.931131,0.902337,0.958786,4.282862,4.240517,92.0,92.0,46000000.0,220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5.0,4.0,5.0,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84.0,95.0,90.0,82.0,86.0,88.0,81.0,76.0,77.0,92.0,87.0,91.0,87.0,95.0,71.0,94.0,95.0,84.0,78.0,93.0,63.0,29.0,95.0,82.0,84.0,95.0,32.0,24.0
2,R. Lewandowski,31.0,184.0,80.0,5.162207,5.256266,0.947527,0.886313,0.980984,0.888229,4.223408,4.186069,91.0,91.0,80000000.0,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4.0,4.0,4.0,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71.0,94.0,85.0,84.0,89.0,85.0,79.0,85.0,70.0,88.0,77.0,78.0,77.0,93.0,82.0,89.0,84.0,76.0,86.0,85.0,81.0,49.0,94.0,79.0,88.0,88.0,42.0,19.0
3,Neymar Jr,28.0,175.0,68.0,2.934016,2.917893,1.044742,0.979750,0.974147,1.053869,4.356846,4.326851,91.0,91.0,90000000.0,270000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,5.0,5.0,5.0,High/Medium,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,85.0,87.0,62.0,87.0,87.0,95.0,88.0,89.0,81.0,95.0,94.0,89.0,96.0,91.0,83.0,80.0,62.0,81.0,50.0,84.0,51.0,36.0,87.0,90.0,92.0,93.0,30.0,29.0
4,K. De Bruyne,29.0,181.0,70.0,5.326507,5.331908,0.947527,0.886313,0.980984,0.888229,4.223408,4.186069,91.0,91.0,87000000.0,370000.0,1.0,0.0,0.0,0.0,1

In [10]:
foot_exploded = emb_players_df['preferred_foot'].explode()

encoder = OneHotEncoder(sparse_output=False)

encoded_foot = encoder.fit_transform(foot_exploded.to_numpy().reshape(-1,1))

foot_col = encoder.get_feature_names_out(['feet'])
player_foot_df = pd.DataFrame(encoded_foot, columns=foot_col, index=foot_exploded.index)
player_foot_df = player_foot_df.groupby(player_foot_df.index).max()

player_df_foot_pos = emb_players_df.columns.get_loc('preferred_foot')
df_before = emb_players_df.iloc[:,:player_df_foot_pos]
df_after = emb_players_df.iloc[:,player_df_foot_pos+1:]

emb_players_df = pd.concat([df_before,player_foot_df,df_after], axis=1)
emb_players_df

,short_name,age,height_cm,weight_kg,nationality_emb_0,nationality_emb_1,club_name_emb_0,club_name_emb_1,club_name_emb_2,club_name_emb_3,league_name_emb_0,league_name_emb_1,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,feet_Left,feet_Right,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33.0,170.0,72.0,5.200541,5.201881,0.923094,0.908502,0.884788,0.884957,4.387769,4.342915,93.0,93.0,67500000.0,560000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,5.0,4.0,4.0,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85.0,95.0,70.0,91.0,88.0,96.0,93.0,94.0,91.0,96.0,91.0,80.0,91.0,94.0,95.0,86.0,68.0,72.0,69.0,94.0,44.0,40.0,93.0,95.0,75.0,96.0,35.0,24.0
1,Cristiano Ronaldo,35.0,187.0,83.0,3.258914,3.233033,0.898334,0.931131,0.902337,0.958786,4.282862,4.240517,92.0,92.0,46000000.0,220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,4.0,5.0,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84.0,95.0,90.0,82.0,86.0,88.0,81.0,76.0,77.0,92.0,87.0,91.0,87.0,95.0,71.0,94.0,95.0,84.0,78.0,93.0,63.0,29.0,95.0,82.0,84.0,95.0,32.0,24.0
2,R. Lewandowski,31.0,184.0,80.0,5.162207,5.256266,0.947527,0.886313,0.980984,0.888229,4.223408,4.186069,91.0,91.0,80000000.0,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,4.0,4.0,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71.0,94.0,85.0,84.0,89.0,85.0,79.0,85.0,70.0,88.0,77.0,78.0,77.0,93.0,82.0,89.0,84.0,76.0,86.0,85.0,81.0,49.0,94.0,79.0,88.0,88.0,42.0,19.0
3,Neymar Jr,28.0,175.0,68.0,2.934016,2.917893,1.044742,0.979750,0.974147,1.053869,4.356846,4.326851,91.0,91.0,90000000.0,270000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,5.0,High/Medium,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,85.0,87.0,62.0,87.0,87.0,95.0,88.0,89.0,81.0,95.0,94.0,89.0,96.0,91.0,83.0,80.0,62.0,81.0,50.0,84.0,51.0,36.0,87.0,90.0,92.0,93.0,30.0,29.0
4,K. De Bruyne,29.0,181.0,70.0,5.326507,5.331908,0.947527,0.886313,0.980984,0.888229,4.223408,4.186069,91.0,91.0,87000000.0,370000.0,1.

In [12]:
players_work_split = emb_players_df['work_rate'].str.split('/').apply(lambda x: [pos.strip() for pos in x])
players_work_exploded = players_work_split.explode()

encoder = OneHotEncoder(sparse_output=False)

encoded_tags = encoder.fit_transform(players_work_exploded.to_numpy().reshape(-1,1))

work_columns = encoder.get_feature_names_out(['work'])
players_work_df = pd.DataFrame(encoded_tags, columns=work_columns, index=players_work_exploded.index)
players_work_df = players_work_df.groupby(players_work_df.index).max()

work_col_pos = emb_players_df.columns.get_loc('work_rate')
df_before = emb_players_df.iloc[:, :work_col_pos]
df_after = emb_players_df.iloc[:, work_col_pos+1:]

emb_players_df = pd.concat([df_before, players_work_df, df_after], axis=1)
emb_players_df

,short_name,age,height_cm,weight_kg,nationality_emb_0,nationality_emb_1,club_name_emb_0,club_name_emb_1,club_name_emb_2,club_name_emb_3,league_name_emb_0,league_name_emb_1,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,feet_Left,feet_Right,international_reputation,weak_foot,skill_moves,work_High,work_Low,work_Medium,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33.0,170.0,72.0,5.200541,5.201881,0.923094,0.908502,0.884788,0.884957,4.387769,4.342915,93.0,93.0,67500000.0,560000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,5.0,4.0,4.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85.0,95.0,70.0,91.0,88.0,96.0,93.0,94.0,91.0,96.0,91.0,80.0,91.0,94.0,95.0,86.0,68.0,72.0,69.0,94.0,44.0,40.0,93.0,95.0,75.0,96.0,35.0,24.0
1,Cristiano Ronaldo,35.0,187.0,83.0,3.258914,3.233033,0.898334,0.931131,0.902337,0.958786,4.282862,4.240517,92.0,92.0,46000000.0,220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,4.0,5.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84.0,95.0,90.0,82.0,86.0,88.0,81.0,76.0,77.0,92.0,87.0,91.0,87.0,95.0,71.0,94.0,95.0,84.0,78.0,93.0,63.0,29.0,95.0,82.0,84.0,95.0,32.0,24.0
2,R. Lewandowski,31.0,184.0,80.0,5.162207,5.256266,0.947527,0.886313,0.980984,0.888229,4.223408,4.186069,91.0,91.0,80000000.0,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,4.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71.0,94.0,85.0,84.0,89.0,85.0,79.0,85.0,70.0,88.0,77.0,78.0,77.0,93.0,82.0,89.0,84.0,76.0,86.0,85.0,81.0,49.0,94.0,79.0,88.0,88.0,42.0,19.0
3,Neymar Jr,28.0,175.0,68.0,2.934016,2.917893,1.044742,0.979750,0.974147,1.053869,4.356846,4.326851,91.0,91.0,90000000.0,270000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,5.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,85.0,87.0,62.0,87.0,87.0,95.0,88.0,89.0,81.0,95.0,94.0,89.0,96.0,91.0,83.0,80.0,62.0,81.0,50.0,84.0,51.0,36.0,87.0,90.0,92.0,93.0,30.0,29.0
4,K. De Bruyne,29.0,181.0,70.0,5.326507,5.331908,0.947527,0.886313,0.980984,0.888229,4.223408,4.186069,91.0,91

In [13]:
emb_players_df.to_csv('final_players_df.csv', index=False)